In [1]:
import grequests 
import requests
import os 
import inspect 
from functools import wraps

In [62]:
class AlphaVantageDecorator:
    _ENDPOINT = 'https://www.alphavantage.co/query?'
    
    def __init__(self, symbols, num_sessions=10):
        self.auth = os.environ.get('ALPHA_ADVANTAGE', None)
        if not self.auth:
            raise ValueError("cannot find alphavantage api key")
        if not isinstance(symbols, list) or len(symbols) == 0:
            raise TypeError("symbols must be a list consists of more than one symbol")
        self.symbols = [symbol.lower() for symbol in symbols]
        self.num_sessions = num_sessions
        self.sessions = [requests.Session() for i in range(self.num_sessions)]

    @classmethod
    def _return_response(cls, func):
        "Decorator for issuing multiple http requests"
        argspec = inspect.getfullargspec(func)
        
        @wraps(func)
        def wrapper(self, *args, **kwargs):
            func_name = func(self, *args, **kwargs)
            params = {'apikey':self.auth, 'function':func_name}
            base_url = AlphaVantageClient._ENDPOINT
            print(argspec.args)
            print(kwargs)
            for arg_name in argspec.args[1:]:
                if arg_name in kwargs and kwargs[arg_name]:
                    params[arg_name] = kwargs[arg_name]
            print(params)
            reqs = []
            i = 0
            for symbol in self.symbols:
                params['symbol'] = symbol
                try:
                    reqs.append(grequests.get(base_url, 
                                              session=self.sessions[i % self.num_sessions], 
                                              params=params))
                    i+=1
                    responses = grequests.imap(reqs, size=self.num_sessions)
                except Exception as e:
                    pass
            if not responses: 
                raise ValueError("empty responses")
            return (r.json() for r in responses if r.status_code==200)
        return wrapper 

In [63]:
class AlphaVantageClient(AlphaVantageDecorator):
    @AlphaVantageDecorator._return_response
    def get_daily(self, outputsize=None):
        return "TIME_SERIES_DAILY"

In [64]:
client = AlphaVantageClient(symbols=['aapl', 'AMZN', 'spot', 'nvda',
                                     'intu', 'hrs', 'ibm', 'msft'])

In [65]:
responses_json = client.get_daily()

['self', 'outputsize']
{}
{'apikey': 'VW8HKFZTQT03O5BB', 'function': 'TIME_SERIES_DAILY'}


In [66]:
for x in responses_json:
    print(x)
    break

{'Meta Data': {'1. Information': 'Daily Prices (open, high, low, close) and Volumes', '2. Symbol': 'msft', '3. Last Refreshed': '2018-04-19', '4. Output Size': 'Compact', '5. Time Zone': 'US/Eastern'}, 'Time Series (Daily)': {'2018-04-19': {'1. open': '96.4400', '2. high': '97.0700', '3. low': '95.3400', '4. close': '96.1100', '5. volume': '23540522'}, '2018-04-18': {'1. open': '96.2200', '2. high': '96.7200', '3. low': '95.5200', '4. close': '96.4400', '5. volume': '21043287'}, '2018-04-17': {'1. open': '95.0000', '2. high': '96.5400', '3. low': '94.8800', '4. close': '96.0700', '5. volume': '26771000'}, '2018-04-16': {'1. open': '94.0700', '2. high': '94.6600', '3. low': '93.4200', '4. close': '94.1700', '5. volume': '20288083'}, '2018-04-13': {'1. open': '94.0500', '2. high': '94.1800', '3. low': '92.4400', '4. close': '93.0800', '5. volume': '23346063'}, '2018-04-12': {'1. open': '92.4300', '2. high': '94.1600', '3. low': '92.4300', '4. close': '93.5800', '5. volume': '26758879'}, 